# Lab 04: Stationarity and ARIMA
**BSAD 8310: Business Forecasting — University of Nebraska at Omaha**

## Objectives
1. Test for stationarity using ADF and KPSS tests
2. Identify differencing order from unit-root tests
3. Plot and interpret ACF and PACF
4. Manually specify and fit an ARIMA model
5. Use auto-ARIMA (pmdarima) for automatic model selection
6. Compare ARIMA/SARIMA with all prior benchmarks on a 24-month test set
7. Run residual diagnostics (Ljung-Box test + residual ACF)

## Dataset
**RSXFS** — US Advance Monthly Retail Trade Survey, retrieved from FRED.

## Key notation
| Symbol | Meaning |
|--------|---------|
| $\phi_i$ | AR coefficient at lag $i$ |
| $\theta_j$ | MA coefficient at lag $j$ |
| $\varepsilon_t$ | White-noise innovation |
| $d$ | Non-seasonal differencing order |
| $D$ | Seasonal differencing order |
| $B$ | Backshift operator: $B y_t = y_{t-1}$ |
| $\Delta y_t$ | First difference: $y_t - y_{t-1}$ |
| $\Delta_{12} y_t$ | Seasonal difference: $y_t - y_{t-12}$ |

---
## Section 1: Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import warnings
from pathlib import Path

# Statsmodels
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.holtwinters import ExponentialSmoothing

warnings.filterwarnings('ignore')
np.random.seed(42)

# --- UNO color palette -------------------------------------------------------
UNO_BLUE  = '#005CA9'
UNO_RED   = '#E41C38'
UNO_GRAY  = '#525252'
UNO_GREEN = '#15803d'

plt.rcParams.update({
    'figure.dpi': 150,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.prop_cycle': plt.cycler(color=[
        UNO_BLUE, UNO_RED, UNO_GREEN, UNO_GRAY
    ]),
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.labelsize': 11,
})

FIGURES = Path('../Figures')
FIGURES.mkdir(exist_ok=True)

print('Setup complete.')

---
## Section 2: Load Data (FRED RSXFS)

In [ ]:
try:
    import pandas_datareader.data as web
    raw = web.DataReader('RSXFS', 'fred', start='2000-01-01', end='2023-12-31')
    y = raw['RSXFS'].dropna()
    y.index = pd.PeriodIndex(y.index, freq='M')
    data_source = 'FRED'
except Exception:
    from statsmodels.datasets import get_rdataset
    ap = get_rdataset('AirPassengers').data
    y = pd.Series(
        ap['value'].values,
        index=pd.period_range('1949-01', periods=len(ap), freq='M'),
        name='AirPassengers'
    )
    data_source = 'AirPassengers (fallback)'

print(f'Source: {data_source}  |  Obs: {len(y)}  |  Period: {y.index[0]} to {y.index[-1]}')

H = 24
train = y.iloc[:-H]
test  = y.iloc[-H:]
print(f'Train: {len(train)} obs  |  Test: {len(test)} obs (H={H})')

---
## Section 3: Stationarity Tests — ADF and KPSS

**ADF:** $H_0$ = unit root (non-stationary). Reject $H_0$ ⟹ stationary.  
**KPSS:** $H_0$ = stationary. Reject $H_0$ ⟹ non-stationary.

In [ ]:
def run_stationarity_tests(series, label='Series'):
    """Run ADF and KPSS tests and print a formatted summary."""
    adf_result  = adfuller(series.dropna(), autolag='AIC')
    kpss_result = kpss(series.dropna(), regression='c', nlags='auto')

    adf_stat, adf_p = adf_result[0], adf_result[1]
    kpss_stat, kpss_p = kpss_result[0], kpss_result[1]

    adf_verdict  = 'STATIONARY' if adf_p  < 0.05 else 'NON-STATIONARY'
    kpss_verdict = 'NON-STATIONARY' if kpss_p < 0.05 else 'STATIONARY'

    print(f'--- {label} ---')
    print(f'  ADF  stat={adf_stat:7.3f}   p={adf_p:.4f}  → {adf_verdict}')
    print(f'  KPSS stat={kpss_stat:7.3f}   p={kpss_p:.4f}  → {kpss_verdict}')
    agree = (adf_verdict == kpss_verdict)
    print(f'  Agreement: {"YES" if agree else "CONFLICTING"}')
    print()
    return adf_verdict

# Test raw series, first difference, seasonal difference
dy    = train.diff().dropna()
d12y  = train.diff(12).dropna()
d1d12 = train.diff(12).diff().dropna()

_ = run_stationarity_tests(train,  label='Raw series y_t')
_ = run_stationarity_tests(dy,     label='First difference Δy_t')
_ = run_stationarity_tests(d12y,   label='Seasonal difference Δ₁₂y_t')
_ = run_stationarity_tests(d1d12,  label='Δ₁₂Δy_t (first + seasonal)')

---
## Section 4: ACF and PACF Plots

We plot ACF/PACF for three versions of the series:
1. Raw (likely non-stationary)
2. First-differenced
3. First + seasonally differenced

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 9))
series_list = [
    (train,  'Raw $y_t$'),
    (dy,     'First difference $\\Delta y_t$'),
    (d1d12,  '$\\Delta_{12}\\Delta y_t$'),
]

for row, (s, label) in enumerate(series_list):
    plot_acf( s.dropna(), lags=36, ax=axes[row, 0], color=UNO_BLUE,
              title=f'ACF — {label}', zero=False)
    plot_pacf(s.dropna(), lags=36, ax=axes[row, 1], color=UNO_RED,
              title=f'PACF — {label}', zero=False, method='ywm')
    for ax in axes[row]:
        ax.axhline(0, color=UNO_GRAY, linewidth=0.5)
        ax.set_xlabel('Lag')

plt.tight_layout()
plt.savefig(FIGURES / 'lecture04_acf_pacf.png', bbox_inches='tight')
plt.show()
print('ACF/PACF figure saved.')

---
## Section 5: Differencing

Based on the stationarity tests above, determine the appropriate differencing order.

In [ ]:
# Visualise original vs. differenced series
fig, axes = plt.subplots(3, 1, figsize=(11, 7), sharex=False)

axes[0].plot(train.to_timestamp(), color=UNO_BLUE, linewidth=1.2)
axes[0].set_title('Raw series $y_t$')

axes[1].plot(dy.to_timestamp(), color=UNO_RED, linewidth=1.2)
axes[1].axhline(0, color=UNO_GRAY, linewidth=0.5, linestyle='--')
axes[1].set_title('First difference $\\Delta y_t$')

axes[2].plot(d1d12.to_timestamp(), color=UNO_GREEN, linewidth=1.2)
axes[2].axhline(0, color=UNO_GRAY, linewidth=0.5, linestyle='--')
axes[2].set_title('First + seasonal difference $\\Delta_{12}\\Delta y_t$')

for ax in axes:
    ax.set_ylabel('Value')

plt.tight_layout()
plt.savefig(FIGURES / 'lecture04_differencing.png', bbox_inches='tight')
plt.show()

---
## Section 6: Manual ARIMA — Identify, Fit, Diagnose

From the ACF/PACF of $\Delta_{12}\Delta y_t$:
- Significant spike at lag 1 in PACF → AR(1) non-seasonal
- Significant spike at lag 12 in ACF → MA(1) seasonal

Tentative model: **SARIMA(1, 1, 0)(0, 1, 1)[12]**

In [ ]:
# Fit manually specified SARIMA
manual_order         = (1, 1, 0)   # (p, d, q)
manual_seasonal_order = (0, 1, 1)  # (P, D, Q)
m = 12

manual_model = SARIMAX(
    train,
    order=manual_order,
    seasonal_order=(*manual_seasonal_order, m),
    trend='c'
)
manual_fit = manual_model.fit(disp=False)

print(manual_fit.summary().tables[0])
print(f'\nAIC = {manual_fit.aic:.2f}   BIC = {manual_fit.bic:.2f}')

# Forecast
manual_fc = manual_fit.forecast(steps=H)
manual_fc.index = test.index

# Residual diagnostics
resid = manual_fit.resid.dropna()
lb = acorr_ljungbox(resid, lags=[10, 20], return_df=True)
print('\nLjung-Box test (H₀: residuals are white noise):')
print(lb.to_string())

fig, axes = plt.subplots(1, 2, figsize=(11, 3.5))
plot_acf( resid, lags=30, ax=axes[0], color=UNO_BLUE, title='Residual ACF', zero=False)
axes[1].plot(resid.values, color=UNO_GRAY, linewidth=0.8)
axes[1].axhline(0, color=UNO_RED, linewidth=0.5, linestyle='--')
axes[1].set_title('Residuals over time')
plt.tight_layout()
plt.savefig(FIGURES / 'lecture04_residuals_manual.png', bbox_inches='tight')
plt.show()

---
## Section 7: Auto-ARIMA

Use `pmdarima.auto_arima` if available; fall back to a small BIC-based grid search.

In [ ]:
try:
    import pmdarima as pm
    auto_model = pm.auto_arima(
        train,
        seasonal=True, m=12,
        d=1, D=1,
        max_p=3, max_q=3, max_P=2, max_Q=2,
        information_criterion='bic',
        stepwise=True,
        suppress_warnings=True,
        error_action='ignore'
    )
    best_order = auto_model.order
    best_seasonal = auto_model.seasonal_order
    auto_fc = pd.Series(
        auto_model.predict(n_periods=H),
        index=test.index
    )
    print(f'auto_arima selected: SARIMA{best_order}{best_seasonal}')
    print(f'BIC = {auto_model.bic():.2f}')
    source = 'pmdarima'

except ImportError:
    # Manual BIC grid search as fallback
    print('pmdarima not available — running BIC grid search...')
    best_bic = np.inf
    best_order = (1, 1, 0)
    best_seasonal = (0, 1, 1)
    for p in [0, 1, 2]:
        for q in [0, 1, 2]:
            for P in [0, 1]:
                for Q in [0, 1]:
                    try:
                        fit = SARIMAX(
                            train,
                            order=(p, 1, q),
                            seasonal_order=(P, 1, Q, 12),
                            trend='c'
                        ).fit(disp=False)
                        if fit.bic < best_bic:
                            best_bic = fit.bic
                            best_order = (p, 1, q)
                            best_seasonal = (P, 1, Q)
                    except Exception:
                        pass
    best_fit = SARIMAX(
        train,
        order=best_order,
        seasonal_order=(*best_seasonal, 12),
        trend='c'
    ).fit(disp=False)
    auto_fc = best_fit.forecast(steps=H)
    auto_fc.index = test.index
    print(f'BIC-selected: SARIMA{best_order}{best_seasonal}  BIC={best_bic:.2f}')
    source = 'grid search'

---
## Section 8: Metrics — All Models vs. Benchmarks

In [ ]:
def compute_metrics(actual, forecast):
    e = actual.values - np.array(forecast)
    rmse = np.sqrt(np.mean(e**2))
    mae  = np.mean(np.abs(e))
    mape = np.mean(np.abs(e / actual.values)) * 100
    return {'RMSE': rmse, 'MAE': mae, 'MAPE (%)': mape}

# --- Benchmarks & prior models -----------------------------------------------
naive_fc  = np.repeat(train.iloc[-1], H)
snaive_fc = np.tile(train.iloc[-12:].values, 2)[:H]
mean_fc   = np.repeat(train.mean(), H)

ar2_fc = AutoReg(train, lags=2, old_names=False).fit().forecast(steps=H).values

hw_fc = ExponentialSmoothing(
    train, trend='add', seasonal='mul',
    seasonal_periods=12, initialization_method='estimated'
).fit(optimized=True).forecast(H).values

forecasts = {
    'Naïve'              : naive_fc,
    'Seasonal naïve'     : snaive_fc,
    'Mean'               : mean_fc,
    'AR(2)'              : ar2_fc,
    'HW mult. (L3)'      : hw_fc,
    'Manual SARIMA'      : manual_fc.values,
    f'Auto-ARIMA ({source})': auto_fc.values,
}

rows = []
for name, fc in forecasts.items():
    row = compute_metrics(test, fc)
    row['Model'] = name
    rows.append(row)

metrics_df = pd.DataFrame(rows).set_index('Model')[['RMSE', 'MAE', 'MAPE (%)']]
metrics_df = metrics_df.sort_values('RMSE')
print(metrics_df.round(2).to_string())

---
## Section 9: Forecast Plot and Residual Diagnostics

In [ ]:
# --- Forecast comparison plot ------------------------------------------------
pred_summary = manual_fit.get_forecast(steps=H)
pi = pred_summary.conf_int(alpha=0.05)
pi.index = test.index

fig, ax = plt.subplots(figsize=(12, 5))
tail = train.iloc[-48:]
ax.plot(tail.to_timestamp(),   color=UNO_BLUE, linewidth=1.4,
        label='Training data')
ax.plot(test.to_timestamp(),   color=UNO_GRAY, linewidth=1.4,
        linestyle='--', label='Actual')
ax.plot(snaive_fc, color=UNO_GRAY, linewidth=1, linestyle=':',
        label='Seasonal naïve')
ax.plot(manual_fc.to_timestamp(), color=UNO_GREEN, linewidth=2,
        label=f'Manual SARIMA{manual_order}{manual_seasonal_order}')
ax.fill_between(
    pi.to_timestamp().index,
    pi.iloc[:, 0].values,
    pi.iloc[:, 1].values,
    color=UNO_GREEN, alpha=0.15, label='95% PI'
)
ax.plot(auto_fc.to_timestamp(), color=UNO_RED, linewidth=1.5,
        linestyle='-.', label=f'Auto-ARIMA ({source})')
ax.axvline(train.index[-1].to_timestamp(), color=UNO_GRAY,
           linestyle='--', linewidth=0.8)
ax.set_title('ARIMA Forecasts vs. Actual')
ax.set_ylabel('Value')
ax.legend(fontsize=9, loc='upper left')
plt.tight_layout()
plt.savefig(FIGURES / 'lecture04_forecasts.png', bbox_inches='tight')
plt.show()

# --- Residual diagnostics full panel -----------------------------------------
fig = manual_fit.plot_diagnostics(figsize=(12, 6))
fig.suptitle('Residual Diagnostics — Manual SARIMA', y=1.01)
plt.tight_layout()
plt.savefig(FIGURES / 'lecture04_diagnostics.png', bbox_inches='tight')
plt.show()

print('All figures saved to Figures/')